# SNP

In [1]:
import pysam
import array
import numpy as np
import pandas as pd

In [2]:
input_bam = "first_chrom.bam"
target_reads = "./first_chrom_target_snp.bam"
untarget_reads = "./first_chrom_untarget_snp.bam"
output_bam = "./first_chrom_premod_snp.bam"
delete_bed = "./snp_delete.bed"
snp_file = "./snp_f.csv"
n_proc = 4
probability = None

In [3]:
# Читаем файл с мутациями
snips = pd.read_csv(snp_file)

# Создаем Bed-файл с интервалами
with pysam.AlignmentFile(input_bam, "rb") as samfile_input, open(delete_bed, "a") as bed:
    chroms = list(set(samfile_input.references) & set(snips["chromosome"]))
    if len(chroms) == 0:
        # Поменять на raise
        print("Target chromosomes are absent in reference\nPlease, check correctness of your csv file or names of contigs")
    else:
        snips_true = snips.query("chromosome in @chroms")
        for snip_index in snips_true.index:
            chr_name, start = snips_true.loc[snip_index, "chromosome"], snips_true.loc[snip_index, "position"]
            stop = start + 1
            bed.write(f"{chr_name}\t{start}\t{stop}\n")

Самтулс

In [4]:
def nucl_changer(read, number_list, position, nucl, quality, comp, probability=None):
    # По умолчанию создаем гетерозиготную мутацию (вероятность равна 0.05) 
    if (type(comp) == str) and (("YC", 1) in read.get_tags()):
        return read
    
    if type(probability) is not np.float64:
        probability = 0.5
        
    if np.random.choice([0,1], size=1, p=[1 - probability, probability]) == 1:
        ind = number_list.index(position - 1)
        indq, length, rc = ind, 0, read.cigartuples
        for cigar_block in rc:
            if cigar_block[0] in [0, 7, 8]:
                length += cigar_block[1]
                if indq < length:
                    break
            elif cigar_block[0] in [1, 4]:
                length += cigar_block[1]
                indq += cigar_block[1]
#             elif cigar_block[0] == 2:
#                 length += cigar_block[1]
#                 indq -= cigar_block[1]

        if indq < 0:
            print("Warning", read.query_name)
        
        # Мутируем!            
        read.query_sequence = read.query_sequence[:indq] + nucl + read.query_sequence[(indq + 1):]
        if read.query_qualities is not None:
            read.query_qualities[indq] = quality
        if type(comp) == str:  # для компаундов
            read.set_tag("YC", 1)
            
    return read

In [5]:
def snp_maker(read, number_list, snips_true, probability):
    for snip_index in snips_true.index:
        chr_name, position, nucl, quality, comp = snips_true.loc[snip_index, "chromosome"], snips_true.loc[snip_index, "position"], snips_true.loc[snip_index, "nucleotide"], snips_true.loc[snip_index, "quality"], snips_true.loc[snip_index, "compaund"]
        probability = snips_true.loc[snip_index, "probability"]
        comp_counter = []
        if (type(comp) == str):
            if comp not in comp_counter:
                comp_counter.append(comp)
                probability = 0.5
            else:
                probability = 1.0
        if (position - 1) in number_list:
            read = nucl_changer(read, number_list, position, nucl, quality, comp, probability)
    return read

In [6]:
# Ready
snips = pd.read_csv(snp_file)

with pysam.AlignmentFile(target_reads, "rb") as samfile_input, pysam.AlignmentFile(output_bam, "wb", template=samfile_input) as samfile_output:
    chroms = list(set(samfile_input.references) & set(snips["chromosome"]))
    if len(chroms) == 0:
        # Поменять на raise
        print("There are no chromosomes in changed file, which needs to be changed")
    else:
        snips_true = snips.query("chromosome in @chroms")
        reads = samfile_input.fetch()
        for read in reads:
            number_list = read.get_reference_positions()
            read_returned = snp_maker(read, number_list, snips_true, probability)
            samfile_output.write(read_returned)

## CNV

In [1]:
import array
import pysam
import numpy as np
import pandas as pd

In [2]:
input_bam = "finka_cnv.bam"
pre_target_reads = "finka_cnv_pre_target_cnv.bam"
target_reads = "finka_cnv_target_cnv.bam"
sorted_target_reads = "finka_cnv_sorted_target_cnv.bam"
delete_bed = "cnv_delete.bed"
output_bam = "finka_cnv_premod_cnv.bam"
cnv_file = "cnv_f.csv"
target_fasta = "./GRCh38_full_analysis_set_plus_decoy_hla.fa"

In [6]:
# Создаем промежуточный bed, чтобы разнести целевой и нецелевой участки 
# (догадываюсь, что у тебя это реализовано иначе, но тем не менее запишу свой вариант)

cnvs = pd.read_csv(cnv_file)

with pysam.AlignmentFile(input_bam, "rb") as samfile_input, open(delete_bed, "w") as bed:
    chroms = list(set(samfile_input.references) & set(cnvs["chromosome"]))
    if len(chroms) == 0:
        # Поменять на raise
        print("Target chromosomes are absent in reference\nPlease, check correctness of your csv file or names of contigs")
    else:
        for chr_name in chroms:  # Проходимся по каждой хромосоме, которая есть и в выравнивании, и в csv
            chrom_subset = cnvs.query("chromosome==@chr_name")
            for cnv_num in chrom_subset.index:
                # Сделать более гибким 1200 (задать, как переменную)
                start, stop = chrom_subset.loc[cnv_num, "position_start"], chrom_subset.loc[cnv_num, "position_finish"]
                bed.write(f"{chr_name}\t{start}\t{stop + 1}\n")

Очень важное замечание: координаты делеций/инсерций:
- Старт -- первый нуклеотид, которого нет / который удвоен
- Стоп -- последний нуклеотид, которого нет / который удвоен

In [7]:
def snp_polisher(read, bad_dict):
    number_list = read.get_reference_positions()
#     print(f"name: {read.query_name}\n\
#     l_bef: {len(read.query_sequence)}\n\
#     c_bef: {read.cigartuples}")

#     ch_p: {bad_dict}\n\         chr: {read.reference_name}\n\     st: {read.reference_start}\n\
    for local_position_ref in bad_dict.keys():
        target_nucl = bad_dict[local_position_ref]   # Добавить условие для делеции
                    
        
        try:  # Для парных ридов без такой позиции
            position = number_list.index(local_position_ref)  # Это уже 0-based, поэтому ничего не меняем
            #if read.query_name == "A00132:134:HVTL3DSXX:2:2277:20871:12023":
#                 print(position)
            old_seq = read.query_sequence
            #print(f"oldseq: {len(old_seq)}")
            if target_nucl == "":    
                read.cigartuples = cigar_del(read, [position])  # АКТИВИРОВАТЬ СТРОКУ
                
            if position < len(old_seq):
                read.query_sequence = old_seq[:position] + target_nucl + old_seq[position + 1:]
            else:
                read.query_sequence = old_seq[:position] + target_nucl
        except ValueError:
            #print("Paired without position")
            continue
    
#     print(f"l_aft: {len(read.query_sequence)}\n\
#     c_aft: {read.cigartuples}")
    return read

In [8]:
def snp_worker(pile, pile_nucls, read_names):
    local_bad_reads = {}
    
    ## Dictionary creation
    pile_dict = {}
    for nucl in pile_nucls:
        if nucl in pile_dict:  
            pile_dict[nucl] += 1
        else:
            pile_dict[nucl] = 1
            
    if max(pile_dict.values())/sum(pile_dict.values()) < 0.85:  # Если доминирующий нуклеотид встречается реже, чем в 85%
        target_nucl = max(pile_dict, key=pile_dict.get)        # Выбираем нуклеотид, который остается
        ######## Тут начинается жесткий костыль, который надо переписать! #########
        if target_nucl != "":
            
        ######## Тут заканчивается жесткий костыль, который надо переписать! #########
            local_position_ref = pile.reference_pos     # Находим позицию снипа
            indices_local_bad = [ind for ind, nuc in enumerate(pile_nucls) if nuc != target_nucl]   # Находим номера ридов с "плохой" заменой
            bad_local_reads = list(read_names[indices_local_bad]) # Получаем список ридов с "плохой заменой"
            local_bad_reads = {read: {local_position_ref: target_nucl} for read in bad_local_reads}
        
    return local_bad_reads

In [9]:
def snp_detector(samfile_input, chr_name, start, stop):
    #print("Detector works")
    bad_reads = {}
    for pile in samfile_input.pileup(chr_name, start, stop):
        read_names = np.array(pile.get_query_names()) # Достаем имена ридов
        pile_nucls = [nucl.upper() for nucl in pile.get_query_sequences()] # Достаем нуклеотиды на позиции (и единообразим регистр)
          # Список ридов, которые стоит удалить
        if len(set(pile_nucls)) > 1:   # Если есть разные нуклетиды на позиции
            local_bad_reads = snp_worker(pile, pile_nucls, read_names)
            for read, feature in local_bad_reads.items():
                if read in bad_reads:
                    bad_reads[read].update(local_bad_reads[read])
                else:
                    bad_reads.update({read: feature})
    #print("Detector stops")        
    return bad_reads

In [10]:
# Считает покрытие в однонуклеотидной позиции
def read_cov_one_nucl(samfile, chr_name, nucl):
    cov_iter = samfile.pileup(chr_name, nucl, nucl + 1, truncate=True)
    for position in cov_iter:
        cov = position.nsegments
    return(cov)


# Сохраняет кусок в +/- размер рида оснований от краев делеции
## Переписать без дублирования кода
## Сделать вменяемо с единицами
def read_del_tails(samfile, chr_name, start, stop, fasta_ref, read_length=150):   # Добавить длину рида в инпут на старте или вычислять из бама
    const = read_length * 7  # В нашем случае - 1050 нуклеотидов
    with pysam.FastaFile(target_fasta) as fasta_ref:
        region_left = fasta_ref.fetch(chr_name, start - const - 1, start - 1) # Отнимаем 1, поскольку нужно не включать первый нуклеотид делеции, формат 1-based
        region_right = fasta_ref.fetch(chr_name, stop, stop + const) # Аналогичная логика
    return region_left, region_right
        
# Клепаем сплит-риды
def create_split_reads_lb(samfile_input, region_left, region_right, counter, chr_name, start, stop, probability, read_length=150):  
    mapq_probs = [1/80 for i in range(20,60)] + [0.5] # Эмпирическое наблюдение разброса качества рида
    # Рандомно выбираем длину софтклипов (10-200 с каждого края)
    min_soft = int(read_length / 25)
    max_soft = int(read_length * 4 / 5)
    right_num = np.random.choice(range(min_soft, max_soft), size=1)[0]
    left_num = read_length - right_num
    
    # Создаем сплит-рид и добавляем его свойства
    new_split_read = pysam.AlignedSegment(header=samfile_input.header) # Надо сделать нормальный хедер
    new_split_read.query_name = f"SRRread_split_l_{counter}"  #? Это норм?
    new_split_read.query_sequence=f"{region_left[-left_num:]}{region_right[:right_num]}"
    new_split_read.reference_name = chr_name
    if chr_name[3:] == "X":
        new_split_read.reference_id = 22
    elif chr_name[3:] == "Y":
        new_split_read.reference_id = 23
    elif type(chr_name[3:]) == int:
        new_split_read.reference_id = int(chr_name[3:]) - 1
    new_split_read.flag = np.random.choice([163, 99, 147, 83], size=1)[0]  #?  Допинфа от Полины и Кати
    new_split_read.mapping_quality = np.random.choice(a=range(20,61), size=1, p=mapq_probs)[0]
    new_split_read.reference_start = start - 1 - left_num # Потому что BAM 0-based
    new_split_read.next_reference_id = new_split_read.reference_id
    new_split_read.cigartuples = [(0, left_num), (4, right_num)]
    new_split_read.query_qualities = array.array('B', np.random.choice(range(18,36), size=len(new_split_read.query_sequence)).tolist())
    new_split_read.tags = [("NM", 1),
          ("RG", "L1")]  #?   Допинфа от Полины и Кати
    
    # Создаем парный ему рид
    
    pair_new_split_read = pysam.AlignedSegment(header=samfile_input.header) # Надо сделать нормальный хедер
    pair_new_split_read.query_name = f"SRRread_split_l_{counter}"  #? Это норм?
    pair_new_split_read.reference_name = chr_name
    pair_new_split_read.reference_id = new_split_read.reference_id

    pair_new_split_read.mapping_quality = new_split_read.mapping_quality
    pair_new_split_read.cigartuples = [(0, read_length)]
    pair_new_split_read.tags = [("NM", 1),
          ("RG", "L1")]  #?   Допинфа от Полины и Кати
    pair_new_split_read.next_reference_id = new_split_read.reference_id

    
    # Если первый рид направлен направо, то:
    if new_split_read.flag in [99, 163]:
        next_start_coeff = np.random.choice(range(52,502), size = 1)[0] # Случайным образом отберем расстояние от второй границы делеции до старта парного рида
        new_split_read.next_reference_start = stop + next_start_coeff
        new_split_read.template_length = new_split_read.next_reference_start + read_length - new_split_read.reference_start
        pair_new_split_read.query_sequence=f"{region_right[next_start_coeff:next_start_coeff+read_length]}"
        if new_split_read.flag == 99:
            pair_new_split_read.flag = 147
        elif new_split_read.flag == 163:
            pair_new_split_read.flag = 83
           
        
    # А если налево, то:    
    elif new_split_read.flag in [83, 147]:
        next_start_coeff = np.random.choice(range(read_length + 2, 552), size = 1)[0] 
        new_split_read.next_reference_start = start - next_start_coeff - read_length - 1
        new_split_read.template_length = - (start + 1 - new_split_read.next_reference_start)
        pair_new_split_read.query_sequence = f"{region_left[ -next_start_coeff - read_length: - next_start_coeff]}"
        if new_split_read.flag == 147:
            pair_new_split_read.flag = 99
        elif new_split_read.flag == 83:
            pair_new_split_read.flag = 163
            
    pair_new_split_read.reference_start = new_split_read.next_reference_start
    try:
        pair_new_split_read.query_qualities = array.array('B', np.random.choice(range(18,36), size=len(pair_new_split_read.query_sequence)).tolist())
    except TypeError:
        print(f"Left boundary, paired flag:{pair_new_split_read.flag}")
        print(next_start_coeff, next_start_coeff + read_length) if pair_new_split_read.flag in [147, 83] else print(- left_num - next_start_coeff, - left_num - next_start_coeff + read_length)
        print(len(region_left), len(region_right))
    pair_new_split_read.next_reference_start = new_split_read.reference_start     
    pair_new_split_read.template_length = - new_split_read.template_length 
#     print(f"first_read: {new_split_read},\
#           second_read: {pair_new_split_read}")
    
    return new_split_read, pair_new_split_read

def create_split_reads_rb(samfile_input, region_left, region_right, counter, chr_name, start, stop, probability, read_length=150):  
    mapq_probs = [1/80 for i in range(20,60)] + [0.5] # Эмпирическое наблюдение разброса качества рида
    min_soft = int(read_length / 25)
    max_soft = int(read_length * 4 / 5)
    left_num = np.random.choice(range(min_soft, max_soft), size=1)[0]
    right_num = read_length - left_num
    
    # Создаем сплит-рид и добавляем его свойства
    new_split_read = pysam.AlignedSegment(header=samfile_input.header) # Надо сделать нормальный хедер
    new_split_read.query_name = f"SRRread_split_r_{counter}"  #? Это норм?
    new_split_read.query_sequence=f"{region_left[-left_num:]}{region_right[:right_num]}"
    new_split_read.reference_name = chr_name
    if chr_name[3:] == "X":
        new_split_read.reference_id = 22
    elif chr_name[3:] == "Y":
        new_split_read.reference_id = 23
    elif type(chr_name[3:]) == int:
        new_split_read.reference_id = int(chr_name[3:]) - 1
    new_split_read.flag = np.random.choice([163, 99, 147, 83], size=1)[0]  #?  Допинфа от Полины и Кати
    new_split_read.mapping_quality = np.random.choice(a=range(20,61), size=1, p=mapq_probs)[0]
    new_split_read.reference_start = stop# - left_num # Потому что BAM 0-based
    new_split_read.next_reference_id = new_split_read.reference_id
    new_split_read.cigartuples = [(4, left_num), (0, right_num)]
    new_split_read.query_qualities = array.array('B', np.random.choice(range(18,36), size=len(new_split_read.query_sequence)).tolist())
    new_split_read.tags = (("NM", 1),
          ("RG", "L1"))  #?   Допинфа от Полины и Кати
    
    # Создаем парный ему рид
    
    pair_new_split_read = pysam.AlignedSegment(header=samfile_input.header) # Надо сделать нормальный хедер
    pair_new_split_read.query_name = f"SRRread_split_r_{counter}"  #? Это норм?
    pair_new_split_read.reference_name = chr_name
    pair_new_split_read.reference_id = new_split_read.reference_id

    pair_new_split_read.mapping_quality = new_split_read.mapping_quality
    pair_new_split_read.cigartuples = [(0, read_length)]
    pair_new_split_read.tags = (("NM", 1),
          ("RG", "L1"))  #?   Допинфа от Полины и Кати
    pair_new_split_read.next_reference_id = new_split_read.reference_id

    
    # Если первый рид направлен направо, то:
    if new_split_read.flag in [99, 163]:
        next_start_coeff = np.random.choice(range(read_length + 2, 500), size = 1)[0] # Случайным образом отберем расстояние от второй границы делеции до старта парного рида
        new_split_read.next_reference_start = stop + right_num + next_start_coeff
        new_split_read.template_length = new_split_read.next_reference_start + read_length - new_split_read.reference_start
        pair_new_split_read.query_sequence=f"{region_right[right_num + next_start_coeff:right_num + next_start_coeff+read_length]}"
        if new_split_read.flag == 99:
            pair_new_split_read.flag = 147
        elif new_split_read.flag == 163:
            pair_new_split_read.flag = 83
           
        
    # А если налево, то:    
    elif new_split_read.flag in [83, 147]:
        next_start_coeff = np.random.choice(range(read_length + 2, 552), size = 1)[0] # Считаем, что риды могут перекрываться не более, чем на 90 нуклеотидов
        new_split_read.next_reference_start = start - next_start_coeff - read_length - 1
        new_split_read.template_length = - (stop + right_num - new_split_read.next_reference_start)
        pair_new_split_read.query_sequence = f"{region_left[-next_start_coeff - read_length: -next_start_coeff]}"
        if new_split_read.flag == 147:
            pair_new_split_read.flag = 99
        elif new_split_read.flag == 83:
            pair_new_split_read.flag = 163
            
    pair_new_split_read.reference_start = new_split_read.next_reference_start
    try:
        pair_new_split_read.query_qualities = array.array('B', np.random.choice(range(18,36), size=len(pair_new_split_read.query_sequence)).tolist())
    except TypeError:
        print(f"Left boundary, paired flag:{pair_new_split_read.flag}")
        print(next_start_coeff, next_start_coeff + read_length) if pair_new_split_read.flag in [147, 83] else print(- left_num - next_start_coeff, - left_num - next_start_coeff + read_length)
        print(len(region_left), len(region_right))
    pair_new_split_read.next_reference_start = new_split_read.reference_start     
    pair_new_split_read.template_length = - new_split_read.template_length 
#     print(f"first_read: {new_split_read},\
#           second_read: {pair_new_split_read}")
    return new_split_read, pair_new_split_read

In [11]:
def decreasing_coverage(samfile_input, samfile_output, chr_name, start, stop, probability, read_length=150):
    reads = samfile_input.fetch(chr_name, start, stop)
    for read in reads:
        if (read.reference_start > start) or (read.reference_start + read_length < stop):
            if np.random.choice([0,1], size=1, p=[1 - probability, probability]) == 0:
                samfile_output.write(read)
        else:
            samfile_output.write(read)

In [12]:
def increasing_coverage(samfile_input, samfile_output, chr_name, start, stop, probability, read_length=150):
    reads = samfile_input.fetch(chr_name, start, stop)
    for read in reads:
        #print(f"Writing read {read.query_name}")
        samfile_output.write(read)
        #print(f"Read {read.query_name} written in a first time")
        if (read.reference_start > start) and (read.reference_start + read_length < stop):
            if np.random.choice([0,1], size=1, p=[1 - probability, probability]) == 1:
                samfile_output.write(read)
                #print(f"Read {read.query_name} written in a second time")

In [13]:
def read_ins_tails(samfile, chr_name, start, stop, fasta_ref, read_length=150):   # ВНИМАНИЕ НА ДЛИНУ РИДА СО ВСЕМИ ВЫТЕКАЮЩИМИ
    const = read_length * 4  # В нашем случае - 1000 нуклеотидов
    with pysam.FastaFile(target_fasta) as fasta_ref:
        region_left = fasta_ref.fetch(chr_name, start - 1, start - 1 + const) # Отнимаем 1, поскольку нужно не включать первый нуклеотид делеции, формат 1-based
        region_right = fasta_ref.fetch(chr_name, stop - const, stop + 1) # Аналогичная логика
    return region_left, region_right

In [14]:
def create_split_ins_reads_lb(samfile_input, region_left, region_right, counter, chr_name, start, stop, probability, read_length=150):
    #print(f"reg_left_right: {region_left}, {region_right}")
    
    mapq_probs = [1/80 for i in range(20,60)] + [0.5] # Эмпирическое наблюдение разброса качества рида
    # Рандомно выбираем длину софтклипов (10-200 с каждого края)
    min_soft = int(read_length / 25)
    max_soft = int(read_length * 4 / 5)
    left_num = np.random.choice(range(min_soft, max_soft), size=1)[0]
    right_num = read_length - left_num
    
    # Создаем сплит-рид и добавляем его свойства
    new_split_read = pysam.AlignedSegment(header=samfile_input.header) # Надо сделать нормальный хедер
    new_split_read.query_name = f"SRRread_split_ins_l_{counter}"  #? Это норм?
    new_split_read.query_sequence=f"{region_right[-left_num:]}{region_left[:right_num]}"   # Делаем все наоборот
    new_split_read.reference_name = chr_name
    if chr_name[3:] == "X":
        new_split_read.reference_id = 22
    elif chr_name[3:] == "Y":
        new_split_read.reference_id = 23
    elif type(chr_name[3:]) == int:
        new_split_read.reference_id = int(chr_name[3:]) - 1
    new_split_read.flag = np.random.choice([163, 99, 147, 83], size=1)[0]  
    new_split_read.mapping_quality = np.random.choice(a=range(20,61), size=1, p=mapq_probs)[0]
    new_split_read.reference_start = start - 1 # Потому что BAM 0-based
    new_split_read.next_reference_id = new_split_read.reference_id
    new_split_read.cigartuples = [(4, left_num), (0, right_num)]
    #print(len(new_split_read.query_sequence), new_split_read.cigartuples)
    new_split_read.query_qualities = array.array('B', np.random.choice(range(18,36), size=len(new_split_read.query_sequence)).tolist())
    new_split_read.tags = [("NM", 1),
          ("RG", "L1")]  #?   Допинфа от Полины и Кати
    
    # Создаем парный ему рид
    
    pair_new_split_read = pysam.AlignedSegment(header=samfile_input.header) # Надо сделать нормальный хедер
    pair_new_split_read.query_name = f"SRRread_split_ins_l_{counter}"  #? Это норм?
    pair_new_split_read.reference_name = chr_name
    pair_new_split_read.reference_id = new_split_read.reference_id

    pair_new_split_read.mapping_quality = new_split_read.mapping_quality
    pair_new_split_read.cigartuples = [(0, read_length)]
    pair_new_split_read.tags = [("NM", 1),
          ("RG", "L1")]  #?   Допинфа от Полины и Кати
    pair_new_split_read.next_reference_id = new_split_read.reference_id

    
    # Если первый рид направлен направо, то:
    if new_split_read.flag in [99, 163]:
        next_start_coeff = np.random.choice(range(100,400), size = 1)[0] # Случайным образом отберем расстояние от второй границы делеции до старта парного рида
        new_split_read.next_reference_start =  start + next_start_coeff - 1
        new_split_read.template_length = new_split_read.next_reference_start - new_split_read.reference_start + read_length
        #print(f"preLbRpair_new: {next_start_coeff}")
        pair_new_split_read.query_sequence=f"{region_left[next_start_coeff:next_start_coeff+read_length]}"
        if new_split_read.flag == 99:
            pair_new_split_read.flag = 147
        elif new_split_read.flag == 163:
            pair_new_split_read.flag = 83
           
        
    # А если налево, то:    
    elif new_split_read.flag in [83, 147]:
        next_start_coeff = np.random.choice(range(100, 400), size = 1)[0] # Считаем, что риды могут перекрываться не более, чем на 90 нуклеотидов
        new_split_read.next_reference_start = stop - left_num - next_start_coeff + 1
        new_split_read.template_length = new_split_read.next_reference_start - new_split_read.reference_start + read_length
        #print(f"preLbLpair_new: {next_start_coeff}")
        pair_new_split_read.query_sequence = f"{region_right[(- left_num - next_start_coeff):(- left_num - next_start_coeff + read_length)]}"
        if new_split_read.flag == 147:
            pair_new_split_read.flag = 99
        elif new_split_read.flag == 83:
            pair_new_split_read.flag = 163
            
    pair_new_split_read.reference_start = new_split_read.next_reference_start
#     print(len(pair_new_split_read.query_sequence), pair_new_split_read.cigartuples)
    try:
        pair_new_split_read.query_qualities = array.array('B', np.random.choice(range(18,36), size=len(pair_new_split_read.query_sequence)).tolist())
    except TypeError:
        print(f"Left boundary, paired flag:{pair_new_split_read.flag}")
        print(next_start_coeff, next_start_coeff + read_length) if pair_new_split_read.flag in [147, 83] else print(- left_num - next_start_coeff, - left_num - next_start_coeff + read_length)
        print(len(region_left), len(region_right))
    pair_new_split_read.next_reference_start = new_split_read.reference_start     
    pair_new_split_read.template_length = - new_split_read.template_length 
#     print(f"first_read: {new_split_read},\
#           second_read: {pair_new_split_read}")
    
    return new_split_read, pair_new_split_read

def create_split_ins_reads_rb(samfile_input, region_left, region_right, counter, chr_name, start, stop, probability, read_length=150):
    #print(f"reg_left_right: {region_left}, {region_right}")
    mapq_probs = [1/80 for i in range(20,60)] + [0.5] # Эмпирическое наблюдение разброса качества рида
    min_soft = int(read_length / 25)
    max_soft = int(read_length * 4 / 5)
    right_num = np.random.choice(range(min_soft, max_soft), size=1)[0]
    left_num = read_length - right_num
    
    # Создаем сплит-рид и добавляем его свойства
    new_split_read = pysam.AlignedSegment(header=samfile_input.header) # Надо сделать нормальный хедер
    new_split_read.query_name = f"SRRread_split_ins_r_{counter}"  #? Это норм?
    new_split_read.query_sequence=f"{region_right[-left_num:]}{region_left[:right_num]}"
    new_split_read.reference_name = chr_name
    if chr_name[3:] == "X":
        new_split_read.reference_id = 22
    elif chr_name[3:] == "Y":
        new_split_read.reference_id = 23
    elif type(chr_name[3:]) == int:
        new_split_read.reference_id = int(chr_name[3:]) - 1
    new_split_read.flag = np.random.choice([163, 99, 147, 83], size=1)[0]  #?  Допинфа от Полины и Кати
    new_split_read.mapping_quality = np.random.choice(a=range(20,61), size=1, p=mapq_probs)[0]
    new_split_read.reference_start = stop - left_num + 1 # Потому что BAM 0-based
    new_split_read.next_reference_id = new_split_read.reference_id
    new_split_read.cigartuples = [(0, left_num), (4, right_num)]
    #print(len(new_split_read.query_sequence), new_split_read.cigartuples)
    try:
        new_split_read.query_qualities = array.array('B', np.random.choice(range(18,36), size=len(new_split_read.query_sequence)).tolist())
    except TypeError:
        print(f"RIght boundary, original {-left_num} {right_num}")
    new_split_read.tags = (("NM", 1),
          ("RG", "L1"))  #?   Допинфа от Полины и Кати
    
    # Создаем парный ему рид
    
    pair_new_split_read = pysam.AlignedSegment(header=samfile_input.header) # Надо сделать нормальный хедер
    pair_new_split_read.query_name = f"SRRread_split_ins_r_{counter}"  #? Это норм?
    pair_new_split_read.reference_name = chr_name
    pair_new_split_read.reference_id = new_split_read.reference_id

    pair_new_split_read.mapping_quality = new_split_read.mapping_quality
    pair_new_split_read.cigartuples = [(0, read_length)]
    pair_new_split_read.tags = (("NM", 1),
          ("RG", "L1"))  #?   Допинфа от Полины и Кати
    pair_new_split_read.next_reference_id = new_split_read.reference_id

    
    # Если первый рид направлен направо, то:
    if new_split_read.flag in [99, 163]:
        next_start_coeff = np.random.choice(range(150, 400), size = 1)[0] # Случайным образом отберем расстояние от второй границы делеции до старта парного рида
        new_split_read.next_reference_start = start + next_start_coeff - 1
        new_split_read.template_length = - (new_split_read.reference_start + left_num - new_split_read.next_reference_start  + 1)
        #print(f"preNpair_new: {next_start_coeff}")
        pair_new_split_read.query_sequence=f"{region_left[next_start_coeff:next_start_coeff+read_length]}"
        if new_split_read.flag == 99:
            pair_new_split_read.flag = 147
        elif new_split_read.flag == 163:
            pair_new_split_read.flag = 83
           
        
    # А если налево, то:    
    elif new_split_read.flag in [83, 147]:
        next_start_coeff = np.random.choice(range(150, 400), size = 1)[0] # Считаем, что риды могут перекрываться не более, чем на 90 нуклеотидов
        new_split_read.next_reference_start = stop - left_num - next_start_coeff + 1
        new_split_read.template_length = - (new_split_read.reference_start + left_num - new_split_read.next_reference_start  + 1)
        #print(f"preNpair_new: {next_start_coeff}")
        pair_new_split_read.query_sequence = f"{region_right[(- left_num - next_start_coeff):(- left_num - next_start_coeff + read_length)]}"
        if new_split_read.flag == 147:
            pair_new_split_read.flag = 99
        elif new_split_read.flag == 83:
            pair_new_split_read.flag = 163
            
    pair_new_split_read.reference_start = new_split_read.next_reference_start
#     print(len(pair_new_split_read.query_sequence), pair_new_split_read.cigartuples)
    try:
        pair_new_split_read.query_qualities = array.array('B', np.random.choice(range(18,36), size=len(pair_new_split_read.query_sequence)).tolist())
    except TypeError:
        print(f"Right boundary, paired flag:{pair_new_split_read.flag}")
        print(next_start_coeff, next_start_coeff + read_length) if pair_new_split_read.flag in [147, 83] else print(- left_num - next_start_coeff, - left_num - next_start_coeff + read_length)
        print(len(region_left), len(region_right))    
    pair_new_split_read.next_reference_start = new_split_read.reference_start     
    pair_new_split_read.template_length = - new_split_read.template_length 
#     print(f"first_read: {new_split_read},\
#           second_read: {pair_new_split_read}")
    return new_split_read, pair_new_split_read

In [15]:
def duplication(samfile_input, samfile_output, chr_name, start, stop, probability, fasta_ref):
    #coverage = (int(sum(map(lambda x: read_cov_one_nucl(samfile_input, chr_name, x), [start, stop])) / 2))  ПОЧЕМУ-ТО ПЕРЕСТАЛО РАБОТАТЬ
    coverage = 40
    split_num = int(coverage/3.5)
    region_left, region_right = read_ins_tails(samfile_input, chr_name, start, stop, fasta_ref)
    print(split_num)
    
    counter = 0
    for split_read in range(split_num):
        #try:
        counter += 1
        new_read1_lb, new_read2_lb = create_split_ins_reads_lb(samfile_input, region_left, region_right, counter, chr_name, start, stop, probability)
        samfile_output.write(new_read1_lb)
        samfile_output.write(new_read2_lb)
        new_read1_rb, new_read2_rb = create_split_ins_reads_rb(samfile_input, region_left, region_right, counter, chr_name, start, stop, probability)
        samfile_output.write(new_read1_rb)
        samfile_output.write(new_read2_rb)
    
    increasing_coverage(samfile_input, samfile_output, chr_name, start, stop, probability)
#     print(samfile_input.count_coverage(chr_name, start, stop, read_callback="nofilter"))
    
    
def deletion(samfile_input, samfile_output, chr_name, start, stop, probability, fasta_ref, lp_number=3):  # Добавить переменную lp_number для количества "парочек"
    
    # Computing coverage, number of split reads and sequence of reference around deletion
    coverage = (int(sum(map(lambda x: read_cov_one_nucl(samfile_input, chr_name, x), [start, stop]))/2))
    split_num = int(coverage/3.5)
    region_left, region_right = read_del_tails(samfile_input, chr_name, start, stop, fasta_ref) # Вычисляем контекст референса до и после делеции
    #print(f"rl, rr: {len(region_left)}, {len(region_right)}")
    print(split_num)
        
    # Creating split reads
    counter = 0
    for split_read in range(split_num):
        #try:
        counter += 1
        new_read1_lb, new_read2_lb = create_split_reads_lb(samfile_input, region_left, region_right, counter, chr_name, start, stop, probability)
        samfile_output.write(new_read1_lb)
        samfile_output.write(new_read2_lb)
        new_read1_rb, new_read2_rb = create_split_reads_rb(samfile_input, region_left, region_right, counter, chr_name, start, stop, probability)
        samfile_output.write(new_read1_rb)
        samfile_output.write(new_read2_rb)
#         except ValueError:
#             print("Zdes' byl error")
#             continue

    
    # Decreasing coverage
    decreasing_coverage(samfile_input, samfile_output, chr_name, start, stop, probability)  

In [14]:
cnvs = pd.read_csv(cnv_file)

with pysam.AlignmentFile(pre_target_reads, "rb") as samfile_input, pysam.AlignmentFile(target_reads, "wb", template=samfile_input) as samfile_output:
    chroms = list(set(samfile_input.references) & set(cnvs["chromosome"]))
    true_cnvs = cnvs.query("chromosome in @chroms")
    for cnv_index in true_cnvs.index:
        cnv_type, chr_name = true_cnvs.loc[cnv_index, "type"], true_cnvs.loc[cnv_index, "chromosome"]
        start, stop = true_cnvs.loc[cnv_index, "position_start"], true_cnvs.loc[cnv_index, "position_finish"]
        #print(f"New CNV: {chr_name} {start} {stop}")
        if cnv_type == "del":
            #print(chr_name, start, stop)
            bad_reads = snp_detector(samfile_input, chr_name, start, stop)
            #print(f"Bad reads: {bad_reads}")
            reads = samfile_input.fetch(chr_name, start, stop)
            #print(f"Reads: {reads}")
            for read in reads:
                if read.query_name in bad_reads.keys():
                    #print(f"Testing {read.query_name}")
                    read = snp_polisher(read, bad_reads[read.query_name])
                    #print(f"Result: {len(read.query_sequence)} and {read.cigartuples}")
                samfile_output.write(read)
                #print(f"Written {read.query_name}")
        if cnv_type == "dup":
            reads = samfile_input.fetch(chr_name, start, stop)
            for read in reads:
                samfile_output.write(read)

Индексируем таргет

In [21]:
with pysam.AlignmentFile(sorted_target_reads, "rb") as samfile_input, pysam.AlignmentFile(output_bam, "wb", template=samfile_input) as samfile_output, pysam.FastaFile(target_fasta) as fasta_ref:
    chroms = list(set(samfile_input.references) & set(cnvs["chromosome"]))
    true_cnvs = cnvs.query("chromosome in @chroms")
    for cnv_index in true_cnvs.index:
        cnv_type, chr_name = true_cnvs.loc[cnv_index, "type"], true_cnvs.loc[cnv_index, "chromosome"]
        start, stop = true_cnvs.loc[cnv_index, "position_start"], true_cnvs.loc[cnv_index, "position_finish"]
        probability = true_cnvs.loc[cnv_index, "probability"]
        if type(probability) is not np.float64:
            probability = 0.5
        if cnv_type == "del":
            #snp_polisher(samfile_input, chr_name, start, stop)
            deletion(samfile_input, samfile_output, chr_name, start, stop, probability, fasta_ref)
        elif cnv_type == "dup":
            duplication(samfile_input, samfile_output, chr_name, start, stop, probability, fasta_ref)

11
11
11
10
14
10


## Frameshift and another microevents

In [1]:
import array
import pysam
import numpy as np
import pandas as pd

In [2]:
input_bam = "first_chrom.bam"
target_reads = "first_chrom_target_fs.bam"
delete_bed = "fs_delete.bed"
output_bam = "first_chrom_premod_fs.bam"
fs_file = "fs_f.csv"

In [5]:
fs = pd.read_csv(fs_file)

with pysam.AlignmentFile(input_bam, "rb") as samfile_input, open(delete_bed, "w") as bed:
    chroms = list(set(samfile_input.references) & set(fs["chromosome"]))
    if len(chroms) == 0:
        # Поменять на raise
        print("Target chromosomes are absent in reference\nPlease, check correctness of your csv file or names of contigs")
    else:
        fs_true = fs.query("chromosome in @chroms")
        for fs_local in fs_true.index:
            chr_name = fs_true.loc[fs_local, "chromosome"]
            if pd.isna(fs_true.loc[fs_local, "position_finish"]):
                start, stop = fs_true.loc[fs_local, "position_start"], fs_true.loc[fs_local, "position_start"] + 1
            else:
                start, stop = fs_true.loc[fs_local, "position_start"], int(fs_true.loc[fs_local, "position_finish"] + 1)
            bed.write(f"{chr_name}\t{start}\t{stop}\n")    

Что-то на самтулсном

In [16]:
def cigar_del(read, ind_list):
#     print(f"del_ind_list: {ind_list}\n\
#         read_name: {read.query_name}")
    cigar_before = read.cigartuples        # Настраиваем строку cigar
    cigar_after, cur_num, flag = [], 0, "before_del"
    for cigartuple in cigar_before:
        if cigartuple[0] in [1, 2, 4, 5]:   # Делеции, инсерции и клипы не учитываются в референсных позициях
            cigar_after.append(cigartuple)
            
        elif cigartuple[0] == 0:
            if flag == "before_del":
                if cur_num + cigartuple[1] < ind_list[0]:
                    cigar_after.append(cigartuple)
                    cur_num += cigartuple[1]
                elif cur_num + cigartuple[1] == ind_list[0]:
                    cigar_after.append(cigartuple)
                    cur_num += cigartuple[1]
                    cigar_after.append((2, len(ind_list)))
                    #cur_num += len(ind_list)
                    flag = "into_del"
                    
                elif cur_num + cigartuple[1] > ind_list[0]:
                    if cur_num + cigartuple[1] - 1 <= ind_list[-1]:
                        cigar_after.append((0, ind_list[0] - cur_num))
                        cigar_after.append((2, len(ind_list)))
                        cur_num += (ind_list[0] - cur_num)
                        #cur_num += len(ind_list)
                        flag = "after_del"
                    elif cur_num + cigartuple[1] - 1 > ind_list[-1]:
                        cigar_after.append((0, ind_list[0] - cur_num))
                        cigar_after.append((2, len(ind_list)))
                        cigar_after.append((0, cigartuple[1] - (ind_list[0] - cur_num) - len(ind_list)))
                        cur_num += cigartuple[1]
                        flag = "after_del"
                        
            elif flag == "into_del":
                if cur_num + cigartuple[1] - 1 <= ind_list[-1]:
                    cur_num += cigartuple[1]
                elif cur_num + cigartuple[1] - 1 > ind_list[-1]:
                    cigar_after.append((0, cur_num + cigartuple[1] - 1 - ind_list[-1]))
                    cur_num += cigartuple[1]
                    
            elif flag == "after_del":
                cigar_after.append(cigartuple)

    return cigar_after

In [7]:
def cigar_ins(read, ins_len, start_index, read_length=150): 
#     if start_index is None:
#         start_index = stop_index - 2
    cigar_before = read.cigartuples
    cigar_after, cur_num, flag = [], 0, "before_ins"
    
    for cigartuple in cigar_before:
        tuplength = cigartuple[1]
        tupletype = cigartuple[0]
        
        if flag == "before_ins":
            if tupletype != 2:
                if cur_num + tuplength < start_index:
                    cigar_after.append(cigartuple)
                    cur_num += tuplength
                    
                elif cur_num + tuplength == start_index:
                    cigar_after.append(cigartuple)
                    cur_num += tuplength
                    if cur_num + ins_len < read_length:
                        cigar_after.append((1, ins_len))
                        cur_num += ins_len
                    else:
                        new_ins_len = read_length - cur_num
                        cigar_after.append((1, new_ins_len))
                        cur_num += new_ins_len
                        break
                    flag = "after_ins"
                    
                elif cur_num + tuplength > start_index:
                    if cur_num + tuplength + ins_len >= read_length:
                        cigar_after.append((tupletype, start_index - cur_num))
                        cur_num += start_index - cur_num
                        if cur_num + ins_len <= read_length:
                            cigar_after.append((1, ins_len))
                            cur_num += ins_len
                            cigar_after.append((tupletype, read_length - cur_num))
                            break
                        else:
                            cigar_after.append((1, read_length - cur_num))
                            break
                        
                    elif cur_num + tuplength + ins_len < read_length:
                        cigar_after.append((tupletype, start_index - cur_num))
                        cigar_after.append((1, ins_len))
                        cigar_after.append((tupletype, tuplength - (start_index - cur_num)))
                        cur_num += tuplength
                        cur_num += ins_len
                        flag = "after_ins"
            else:
                cigar_after.append(cigartuple)
                    
        elif flag == "after_ins":
            if tupletype != 2:
                if cur_num + tuplength <= read_length:
                    cigar_after.append(cigartuple)
                    cur_num += tuplength
                else:
                    cigar_after.append((tupletype, read_length - cur_num))
                    break
            else:
                cigar_after.append(cigartuple)

    return cigar_after

In [8]:
def microdeletion(read, number_list, chr_name, start, stop, probability, del_len, read_length=150):
    if np.random.choice([0,1], size=1, p=[1 - probability, probability]) == 1:
        ind_list = []
        for nucl in range(start, stop): # проходимся по нуклеотидам
            if (nucl - 1) in number_list:
                nucl_index = number_list.index(nucl - 1) # 0-based позиция нуклеотида в риде
                if (nucl_index > (4 + del_len * 2)) and (nucl_index < (read_length - (4 + del_len * 2))):
                    print(read.query_name, nucl_index, del_len)
                    ind_list.append(nucl_index)
                
        if len(ind_list) == 0:
            return(read)
        
        read.cigartuples = cigar_del(read, ind_list)

        if ind_list[-1] < read.query_length - 1:  # Если последний нуклеотид из делетированных не на конце рида
            read.query_sequence = read.query_sequence[:ind_list[0]] + read.query_sequence[(ind_list[-1]+1):]              
        else:
            read.query_sequence = read.query_sequence[:ind_list[0]]
    print(f"del: {read.query_name}\n\
            len: {len(read.query_sequence)}\n\
            cigar: {sum([i[1] for i in read.cigartuples])}, {read.cigartuples}\n\
            start: {read.reference_start}")
    return read
    
    
def microinsertion(read, number_list, chr_name, start, stop, seq, probability, read_length=150):
    if np.random.choice([0,1], size=1, p=[1 - probability, probability]) == 1:
        ins_len = len(seq)
        if (start - 1 > number_list[0]) and (stop < number_list[-1]):   # В этой нижележащей строчке поменял старт на стоп
            start_index, stop_index = number_list.index(start), number_list.index(stop)
            if (start_index > 4 + ins_len * 2) and (stop_index < (read_length - (4 + ins_len * 2))):
                read.query_sequence = read.query_sequence[:start_index] + seq + read.query_sequence[start_index:]
                read.cigartuples = cigar_ins(read, ins_len, start_index)
            else:
                return read
#         elif (start - 1 > number_list[0]):
#             start_index = number_list.index(start)
#             read.query_sequence = read.query_sequence[:start_index] + seq
#             read.cigartuples = cigar_ins(read, ins_len, start_index)
        if len(read.query_sequence) > read_length:
            read.query_sequence = read.query_sequence[:read_length]
    print(f"ins: {read.query_name}\n\
        len: {len(read.query_sequence)}\n\
        cigar: {sum([i[1] for i in read.cigartuples])}, {read.cigartuples}\n\
        start: {read.reference_start},\n\
        flag: {read.flag}")
    return read

In [9]:
def microchanger(read, number_list, fs_true):  # Вместо mode в перспективе будет колонка в csv для каждого варианта
    for fs_local in fs_true.index:
        chr_name, fs_type, start, seq = fs_true.loc[fs_local, "chromosome"], fs_true.loc[fs_local, "type"], fs_true.loc[fs_local, "position_start"], fs_true.loc[fs_local, "sequence"]
        probability = fs_true.loc[fs_local, "probability"]
        if type(probability) is not np.float64:
            probability = 0.5
        if pd.isna(fs_true.loc[fs_local, "position_finish"]):
            stop = fs_true.loc[fs_local, "position_start"] + 1
        else:
            stop = int(fs_true.loc[fs_local, "position_finish"] + 1)
            
        if (start in number_list) or ((stop - 1) in number_list): # Так как размер делеций предполагается меньше размера рида, если часть делеции находится на риде, то либо начало, либо конец обязательно попадут на рид
            if fs_type == "del":
                del_len = stop - start
                read = microdeletion(read, number_list, chr_name, start, stop, probability, del_len)
            elif fs_type == "ins":
                read = microinsertion(read, number_list, chr_name, start, stop, seq, probability)
    return read

In [10]:
fs = pd.read_csv(fs_file)

with pysam.AlignmentFile(target_reads, "rb") as samfile_input, pysam.AlignmentFile(output_bam, "wb", template=samfile_input) as samfile_output:
    chroms = list(set(samfile_input.references) & set(fs["chromosome"]))
    fs_true = fs.query("chromosome in @chroms")
    reads = samfile_input.fetch()
    for read in reads:
        number_list = read.get_reference_positions()
        read_returned = microchanger(read, number_list, fs_true)
        samfile_output.write(read_returned)

ins: A00132:134:HVTL3DSXX:1:1614:15826:10332
        len: 150
        cigar: 150, [(0, 150)]
        start: 61403966,
        flag: 99
ins: A00217:175:HVVTKDSXX:1:1273:13422:34726
        len: 150
        cigar: 150, [(0, 150)]
        start: 61403978,
        flag: 83
ins: A00132:134:HVTL3DSXX:1:2656:17951:4022
        len: 150
        cigar: 150, [(0, 131), (1, 6), (0, 13)]
        start: 61403984,
        flag: 163
ins: A00132:134:HVTL3DSXX:2:1418:4526:9298
        len: 150
        cigar: 150, [(0, 122), (1, 6), (0, 22)]
        start: 61403993,
        flag: 99
ins: A00217:175:HVVTKDSXX:4:2236:1515:10786
        len: 150
        cigar: 150, [(0, 120), (1, 6), (0, 24)]
        start: 61403995,
        flag: 147
ins: A00132:134:HVTL3DSXX:2:2330:1208:35994
        len: 150
        cigar: 150, [(0, 116), (1, 6), (0, 28)]
        start: 61403999,
        flag: 163
ins: A00217:175:HVVTKDSXX:2:2432:4453:33661
        len: 150
        cigar: 150, [(0, 106), (1, 6), (0, 38)]
        start: 

A00132:134:HVTL3DSXX:2:1527:19190:16971 58 1
del_ind_list: [58]
        read_name: A00132:134:HVTL3DSXX:2:1527:19190:16971
del: A00132:134:HVTL3DSXX:2:1527:19190:16971
            len: 149
            cigar: 150, [(0, 58), (2, 1), (0, 91)]
            start: 158093329
del: A00132:134:HVTL3DSXX:4:1114:19452:35837
            len: 150
            cigar: 150, [(0, 150)]
            start: 158093329
A00132:134:HVTL3DSXX:4:2403:6370:21324 52 1
del_ind_list: [52]
        read_name: A00132:134:HVTL3DSXX:4:2403:6370:21324
del: A00132:134:HVTL3DSXX:4:2403:6370:21324
            len: 149
            cigar: 150, [(0, 52), (2, 1), (0, 97)]
            start: 158093335
A00132:134:HVTL3DSXX:4:2403:7301:21277 52 1
del_ind_list: [52]
        read_name: A00132:134:HVTL3DSXX:4:2403:7301:21277
del: A00132:134:HVTL3DSXX:4:2403:7301:21277
            len: 149
            cigar: 150, [(0, 52), (2, 1), (0, 97)]
            start: 158093335
del: A00132:134:HVTL3DSXX:1:2364:19190:32002
            len: 150
   